In [3]:
%cd /Users/trancatkhanh/Downloads/GAME
from base.Splendor_v3.env import *

/Users/trancatkhanh/Downloads/GAME


In [15]:
num_game = 100000

In [2]:
from numba import njit
from numba.typed import List
import numba
import numpy as np

In [17]:
@njit
def toNumpy(a):
    lst = np.zeros((len(a),getStateSize()))
    for i in range(len(a)):
        lst[i] = a[i]
    return lst

In [18]:
@njit
def agentPolicy(state,per):
    list_action = getValidActions(state)
    list_action*=per[1]
    action = np.argmax(list_action)
    return action,per

@njit
def agentPolicy2(state,per):
    list_action = getValidActions(state)
    if np.sum(per[3]==state) > np.sum(per[4]==state):
        list_action*=per[2]
    else:
        list_action*=per[1]
    action = np.argmax(list_action)
    return action,per



In [19]:
@njit
def agentPolicyBest(state,per):
    list_action = getValidActions(state)
    list_action*=per[0]
    action = np.argmax(list_action)
    return action,per

@njit
def agentPolicyBest2(state,per):
    list_action = getValidActions(state)
    if np.sum(per[3]==state) > np.sum(per[4]==state):
        list_action*=per[2]
    else:
        list_action*=per[0]
    
    action = np.argmax(list_action)
    return action,per

In [20]:
@njit
def agentRandom(state,per):
    return np.random.choice(np.where(getValidActions(state)==1)[0]),per

In [21]:
@njit
def agentX(state,per):
    if per[7][0][0][0]==0:#step 1,2
        list_action = getValidActions(state)
        list_action*=per[0][0][0]
        action = np.argmax(list_action)
        if getReward(state)!=-1:
            per[2][0][0][0]+=1
            if getReward(state)==1:
                per[1][0][0]+=per[0][0][0]
                per[4][0][0][0]+=1
            else:
                per[0][0][0] = np.random.rand(getActionSize())
            if per[2][0][0][0]%num_game==0:
                per_test = [per[3][0][0],per[1][0][0]]#best, current
                rank = numba_main(agentPolicyBest,agentPolicy,agentRandom,agentRandom,10000,per_test)[0]
                per[4][0][0][0] = rank[1]
                per[5][0][0][0] = rank[0]
                if per[4][0][0][0]>per[5][0][0][0]:
                    per[3][0][0] = per[1][0][0]
                    per[6][0][0][0] = 0
                else:
                    per[6][0][0][0]+=1
                per[1][0][0] = np.zeros(getActionSize())
            if per[6][0][0][0]==3:
                per[7][0][0][0] = 1 #go to step 3
                print('Step 1,2 completed!')
    elif per[7][0][0][0]==1:#step 3
        list_action = getValidActions(state)
        list_action*=per[3][0][0]
        action = np.argmax(list_action)
        per[10].append(state.reshape(1,-1))
        if len(per[8])<50000 and len(per[9]) < 50000:
            if getReward(state)==1:
                for i in per[10]:
                    per[8].append(i)
                per[10].clear()
            if getReward(state)==0:
                for i in per[10]:
                    per[9].append(i)
                per[10].clear()
        else:# find (x) property in state win and loss
            print('Step 3 completed!')
            per[7][0][0][0] = 2
            state_win = toNumpy(per[8])
            state_loss = toNumpy(per[9])
            win_id = np.zeros(getStateSize())
            loss_id = np.zeros(getStateSize())
            state_win1 = state_win.T
            for i in range(getStateSize()):
                uni = np.unique(state_win1[i])
                max_count = -1000
                for j in uni:
                    count1 = np.sum(state_win1[i]==j)
                    if count1 > max_count:
                        max_count = count1
                        win_id[i] = j
            state_loss1 = state_loss.T
            for i in range(getStateSize()):
                uni = np.unique(state_loss1[i])
                max_count = -1000
                for j in uni:
                    count1 = np.sum(state_loss1[i]==j)
                    if count1 > max_count:
                        max_count = count1
                        loss_id[i] = j
            per[11][0][0] = win_id
            per[12][0][0] = loss_id
    else: #step 5,6
        list_action = getValidActions(state)
        if np.sum(per[11][0][0]==state) > np.sum(per[12][0][0]==state):
            list_action*=per[3][0][0]
        else:
            list_action*=per[0][0][0]
        action = np.argmax(list_action)
        if getReward(state)!=-1:
            per[2][0][0][0]+=1
            if getReward(state)==1:
                per[1][0][0]+=per[0][0][0]
                per[4][0][0][0]+=1
            else:
                per[0][0][0] = np.random.rand(getActionSize())
            if per[2][0][0][0]%num_game==0:
                per_test = [per[13][0][0],per[1][0][0],per[3][0][0],per[11][0][0],per[12][0][0]]#best, current, best123, win_fre, loss_fre
                rank = numba_main(agentPolicyBest2,agentPolicy2,agentRandom,agentRandom,10000,per_test)[0]
                per[4][0][0][0] = rank[1]
                per[5][0][0][0] = rank[0]
                if per[4][0][0][0]>per[5][0][0][0]:
                    per[13][0][0] = per[1][0][0]
                    per[14][0][0][0] = 0
                else:
                    per[14][0][0][0]+=1
                per[1][0][0] = np.zeros(getActionSize())
            if per[14][0][0][0]==3:
                raise Exception('Training Completed! Per Saved!')




        

    return action,per

In [4]:
perx = [List([np.random.rand(1,getActionSize())]), #random bias
        List([np.zeros((1,getActionSize()))]), #policy all current 100k game
        List([np.zeros((1,1))]), #game count
        List([np.zeros((1,getActionSize()))]),#best policy so far
        List([np.zeros((1,1))]), #win count
        List([np.zeros((1,1))]), #best win rate count
        List([np.zeros((1,1))]), #check if 3 next best bias is weaker
        List([np.zeros((1,1))]), #mode
        List.empty_list(numba.types.Array(dtype=numba.float64,ndim=2,layout='C')),#state win
        List.empty_list(numba.types.Array(dtype=numba.float64,ndim=2,layout='C')),#state lose
        List.empty_list(numba.types.Array(dtype=numba.float64,ndim=2,layout='C')), #state one game
        List([np.zeros((1,getStateSize()))]), # most frequent win state
        List([np.zeros((1,getStateSize()))]), #most frequent lose state
        List([np.zeros((1,getActionSize()))]), #best policy so far step 4,5,6
        List([np.zeros((1,1))])] #check if 3 training completed
        
        


In [7]:
perx[0][0][0]

array([0.03552749, 0.31564942, 0.56819383, 0.61240635, 0.55556295,
       0.49831801, 0.13699566, 0.27107561, 0.79464359, 0.67033525,
       0.96816143, 0.71792136, 0.34276623, 0.41046133, 0.40800011])

In [23]:
numba_main_2(agentX,10000000,perx,0)[0]

Step 1,2 completed!
Step 3 completed!


Exception: Training Completed! Per Saved!

In [25]:
numba_main_2(agentPolicyBest2,1000,[perx[13][0][0],perx[1][0][0],perx[3][0][0],perx[11][0][0],perx[12][0][0]],0)[0]

571